In [38]:
import pandas as pd
import numpy as np
import os
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.seasonal import seasonal_decompose

In [2]:
train = pd.read_csv('created_train.csv',index_col="id")
test = pd.read_csv('created_test.csv')
holidays = pd.read_csv(f'{os.getcwd()}/extraData/holidays_events.csv')
oil = pd.read_csv(f'{os.getcwd()}/extraData/oil.csv')
stores = pd.read_csv(f'{os.getcwd()}/extraData/stores.csv')
transactions = pd.read_csv(f'{os.getcwd()}/extraData/transactions.csv')

In [85]:
train

,date,store_nbr,family,sales,onpromotion,month-day
id,,,,,,
1297296,2015-01-01,1,AUTOMOTIVE,0.000,0,01-01
1297297,2015-01-01,1,BABY CARE,0.000,0,01-01
1297298,2015-01-01,1,BEAUTY,0.000,0,01-01
1297299,2015-01-01,1,BEVERAGES,0.000,0,01-01
1297300,2015-01-01,1,BOOKS,0.000,0,01-01
...,...,...,...,...,...,...
2974153,2017-07-31,9,POULTRY,470.513,1,07-31
2974154,2017-07-31,9,PREPARED FOODS,61.940,0,07-31
2974155,2017-07-31,9,PRODUCE,1517.552,6,07-31


In [3]:
train = train.iloc[:,1:]

In [4]:
train["date"] = pd.to_datetime(train["date"])
train["month-day"] = train.date.dt.strftime("%m-%d")

### **Outliers** 

In [5]:
#Testing how much the earthquake impacted
earth = train.loc[(train.date.dt.month == 4) & (train.date.dt.year == 2016)].groupby(by="date")["sales"].sum().reset_index()
earth["change"] = (earth['sales'] - earth['sales'].shift(1)) / earth['sales'].shift(1) * 100
earth

,date,sales,change
0,2016-04-01,8.724673e+05,NaN
1,2016-04-02,1.150825e+06,31.904606
2,2016-04-03,1.266908e+06,10.087003
3,2016-04-04,7.952372e+05,-37.230090
4,2016-04-05,7.257501e+05,-8.737908
5,2016-04-06,7.447848e+05,2.622768
6,2016-04-07,5.879397e+05,-21.059119
7,2016-04-08,6.897272e+05,17.312570
8,2016-04-09,9.279814e+05,34.543250
9,2016-04-10,1.008877e+06,8.717346


### **How promotions impact sales?**

In [6]:
train.loc[:,["sales","onpromotion"]].corr()

,sales,onpromotion
sales,1.000000,0.485601
onpromotion,0.485601,1.000000


### **How holidays impact sales?**

In [7]:
holidays = holidays.loc[holidays.transferred == False]
holidays["date"] = pd.to_datetime(holidays["date"])

In [8]:
#Holiday sales to discover in wich holiday types we sell more
#Getting total sales by day
#Just using complete years to get sum by day of year in the last 2 years

grouped = train.loc[train.date.dt.year < 2017].groupby(by="month-day")["sales"].sum().reset_index()
grouped["volume"] = (grouped["sales"] / grouped["sales"].sum())*100
grouped

,month-day,sales,volume
0,01-01,2.920701e+04,0.005516
1,01-02,1.724441e+06,0.325652
2,01-03,1.875616e+06,0.354201
3,01-04,1.686881e+06,0.318559
4,01-05,1.404588e+06,0.265249
...,...,...,...
360,12-27,1.680190e+06,0.317296
361,12-28,1.741219e+06,0.328821
362,12-29,1.764870e+06,0.333287
363,12-30,2.193687e+06,0.414267


In [9]:
holidays["month-day"] = holidays.date.dt.strftime("%m-%d")

In [10]:
yearHols = holidays.drop_duplicates(subset=["month-day"])

In [11]:
yearHols = yearHols.loc[:,["month-day","type"]]

In [12]:
pd.merge(grouped,yearHols,on='month-day', how='left').head(5)

,month-day,sales,volume,type
0,01-01,2.920701e+04,0.005516,Holiday
1,01-02,1.724441e+06,0.325652,Bridge
2,01-03,1.875616e+06,0.354201,NaN
3,01-04,1.686881e+06,0.318559,NaN
4,01-05,1.404588e+06,0.265249,Work Day


In [13]:
#We got the sales volume by each day type so we can encode this categorical values into logical numbers and add them to the train
pd.merge(
    grouped,yearHols,on='month-day', how='left').groupby(
        by="type",dropna=False)["volume"].sum().reset_index().sort_values(
            by="volume",ascending=False)



,type,volume
6,NaN,69.056973
3,Holiday,12.101910
2,Event,10.965209
0,Additional,2.937551
4,Transfer,2.245892
1,Bridge,1.481032
5,Work Day,1.211434


In [14]:
holidayDict = {"Work Day":1,
               "Bridge":2,
               "Transfer":3,
               "Additional":4,
               "Event":5,
               "Holiday":6,
               np.nan:0}

In [15]:
df = pd.merge(train.reset_index(),yearHols.loc[:,["month-day","type"]],on="month-day",how="left")
df["holidayImportance"] = df["type"].map(holidayDict)
df.drop("type",axis=1,inplace=True)
df.head(3)

,id,date,store_nbr,family,sales,onpromotion,month-day,holidayImportance
0,1297296,2015-01-01,1,AUTOMOTIVE,0.0,0,01-01,6
1,1297297,2015-01-01,1,BABY CARE,0.0,0,01-01,6
2,1297298,2015-01-01,1,BEAUTY,0.0,0,01-01,6


In [16]:
#volume refers to volume of total sales in that specific years of the last 2 years
#now we have the holiday importance and the sales volume per day of the year

df = pd.merge(df,grouped.drop("sales",axis=1),on="month-day",how="left")
df.head(5)

,id,date,store_nbr,family,sales,onpromotion,month-day,holidayImportance,volume
0,1297296,2015-01-01,1,AUTOMOTIVE,0.0,0,01-01,6,0.005516
1,1297297,2015-01-01,1,BABY CARE,0.0,0,01-01,6,0.005516
2,1297298,2015-01-01,1,BEAUTY,0.0,0,01-01,6,0.005516
3,1297299,2015-01-01,1,BEVERAGES,0.0,0,01-01,6,0.005516
4,1297300,2015-01-01,1,BOOKS,0.0,0,01-01,6,0.005516


In [17]:
len(df)

1676862

### **How oil prices impact sales?**

In [18]:
oil.rename(columns={"dcoilwtico":"oilPrice"},inplace=True)
oil.ffill(inplace=True)
oil.bfill(inplace=True)
oil["date"] = pd.to_datetime(oil["date"])


In [19]:
oil.head(3)

,date,oilPrice
0,2013-01-01,93.14
1,2013-01-02,93.14
2,2013-01-03,92.97


In [20]:
df = pd.merge(df,oil,on="date",how="left")
df.head(3)

,id,date,store_nbr,family,sales,onpromotion,month-day,holidayImportance,volume,oilPrice
0,1297296,2015-01-01,1,AUTOMOTIVE,0.0,0,01-01,6,0.005516,53.45
1,1297297,2015-01-01,1,BABY CARE,0.0,0,01-01,6,0.005516,53.45
2,1297298,2015-01-01,1,BEAUTY,0.0,0,01-01,6,0.005516,53.45


In [21]:
df["oilPrice"].ffill(inplace=True)

### **How stores impact sales?**

In [22]:
stores.head(3)

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8


In [23]:
stores["type"] = stores.type.map({"A":1,
                                  "B":2,
                                  "C":3,
                                  "D":4,
                                  "E":5})

In [24]:
df = pd.merge(df,stores.loc[:,["store_nbr","type","cluster"]],on="store_nbr",how="left")
df.head(3)

,id,date,store_nbr,family,sales,onpromotion,month-day,holidayImportance,volume,oilPrice,type,cluster
0,1297296,2015-01-01,1,AUTOMOTIVE,0.0,0,01-01,6,0.005516,53.45,4,13
1,1297297,2015-01-01,1,BABY CARE,0.0,0,01-01,6,0.005516,53.45,4,13
2,1297298,2015-01-01,1,BEAUTY,0.0,0,01-01,6,0.005516,53.45,4,13


### **How paying date impact sales?**

In [25]:
df["paying_day"] = ((df["date"].dt.day == 15) | (df["date"].dt.is_month_end)).astype(int)
df.head(5)

,id,date,store_nbr,family,sales,onpromotion,month-day,holidayImportance,volume,oilPrice,type,cluster,paying_day
0,1297296,2015-01-01,1,AUTOMOTIVE,0.0,0,01-01,6,0.005516,53.45,4,13,0
1,1297297,2015-01-01,1,BABY CARE,0.0,0,01-01,6,0.005516,53.45,4,13,0
2,1297298,2015-01-01,1,BEAUTY,0.0,0,01-01,6,0.005516,53.45,4,13,0
3,1297299,2015-01-01,1,BEVERAGES,0.0,0,01-01,6,0.005516,53.45,4,13,0
4,1297300,2015-01-01,1,BOOKS,0.0,0,01-01,6,0.005516,53.45,4,13,0


### **How transaction impact sales?**

In [26]:
transactions["date"] = pd.to_datetime(transactions["date"])
transactions.head(3)

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358


Testing how accurate the data is

In [27]:
t = pd.merge(df,transactions,on=["date","store_nbr"],how="left")
t.head(5)

,id,date,store_nbr,family,sales,onpromotion,month-day,holidayImportance,volume,oilPrice,type,cluster,paying_day,transactions
0,1297296,2015-01-01,1,AUTOMOTIVE,0.0,0,01-01,6,0.005516,53.45,4,13,0,NaN
1,1297297,2015-01-01,1,BABY CARE,0.0,0,01-01,6,0.005516,53.45,4,13,0,NaN
2,1297298,2015-01-01,1,BEAUTY,0.0,0,01-01,6,0.005516,53.45,4,13,0,NaN
3,1297299,2015-01-01,1,BEVERAGES,0.0,0,01-01,6,0.005516,53.45,4,13,0,NaN
4,1297300,2015-01-01,1,BOOKS,0.0,0,01-01,6,0.005516,53.45,4,13,0,NaN


In [28]:
# how could there be stores with no transactions and sales
#Becuase of this I won't use transactions
#also other reason if because transactions data is at a store level and train is at a store-family level
t.loc[(t.transactions ==0) & (t.sales != 0)].head(5)

,id,date,store_nbr,family,sales,onpromotion,month-day,holidayImportance,volume,oilPrice,type,cluster,paying_day,transactions


### **Wich stores are better?**

In [29]:
#By default some store migth sell more because of the location, however bc we don't have coordinates we'll just do a store rank by sales

storeSales = train.groupby(by="store_nbr")["sales"].sum().reset_index().sort_values(by="sales",ascending=False)
storeSales["storeRank"] = [i+1 for i in range(len(storeSales))]
storeSales.head(3)

,store_nbr,sales,storeRank
43,44,3.976564e+07,1
44,45,3.556391e+07,2
2,3,3.322420e+07,3


In [30]:
df = pd.merge(df,storeSales.loc[:,["store_nbr","storeRank"]],on="store_nbr",how="left")

### **Preparing df for modeling**

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1676862 entries, 0 to 1676861
Data columns (total 14 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   id                 1676862 non-null  int64         
 1   date               1676862 non-null  datetime64[ns]
 2   store_nbr          1676862 non-null  int64         
 3   family             1676862 non-null  object        
 4   sales              1676862 non-null  float64       
 5   onpromotion        1676862 non-null  int64         
 6   month-day          1676862 non-null  object        
 7   holidayImportance  1676862 non-null  int64         
 8   volume             1676862 non-null  float64       
 9   oilPrice           1676862 non-null  float64       
 10  type               1676862 non-null  int64         
 11  cluster            1676862 non-null  int64         
 12  paying_day         1676862 non-null  int64         
 13  storeRank          1676862 

In [32]:
df["family"] = df.family.map(
    
    {k:v+1 for k,v in zip(
    df.family.unique(),
    range(len(df.family.unique()))
)})

In [33]:
df.drop(columns=["month-day"],inplace=True)

## **Modeling**

In [48]:
df.head(5)

,id,date,store_nbr,family,sales,onpromotion,holidayImportance,volume,oilPrice,type,cluster,paying_day,storeRank
0,1297296,2015-01-01,1,1,0.0,0,6,0.005516,53.45,4,13,0,34
1,1297297,2015-01-01,1,2,0.0,0,6,0.005516,53.45,4,13,0,34
2,1297298,2015-01-01,1,3,0.0,0,6,0.005516,53.45,4,13,0,34
3,1297299,2015-01-01,1,4,0.0,0,6,0.005516,53.45,4,13,0,34
4,1297300,2015-01-01,1,5,0.0,0,6,0.005516,53.45,4,13,0,34


#### Decomposition

In [44]:
df_ = df.set_index(["date","store_nbr","family"])
freq = 'D'
deco = seasonal_decompose(df_['sales'], period=7)

In [65]:
df_["id"].values

array([1297296, 1297297, 1297298, ..., 2974155, 2974156, 2974157])

In [83]:
res = deco.trend + deco.seasonal + deco.resid
res.reset_index().set_index(df_["id"].values)

,date,store_nbr,family,0
1297296,2015-01-01,1,1,NaN
1297297,2015-01-01,1,2,NaN
1297298,2015-01-01,1,3,NaN
1297299,2015-01-01,1,4,0.000
1297300,2015-01-01,1,5,0.000
...,...,...,...,...
2974153,2017-07-31,9,29,470.513
2974154,2017-07-31,9,30,61.940
2974155,2017-07-31,9,31,NaN
2974156,2017-07-31,9,32,NaN


In [84]:
df.loc[df.id == 2974157]

,id,date,store_nbr,family,sales,onpromotion,holidayImportance,volume,oilPrice,type,cluster,paying_day,storeRank
1676861,2974157,2017-07-31,9,33,12.0,0,0,0.319715,50.21,2,6,1,13
